**Import Libraries**

In [1]:
import pandas as pd
import numpy as np
import datetime as dt

**Read Dataset**

In [11]:
df = pd.read_csv("Retail_Data.csv")
df

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN
...,...,...,...,...,...,...,...
461768,539991,21618,4 WILDFLOWER BOTANICAL CANDLES,1,2010-12-23 16:49:00,1.25,NaN
461769,539991,72741,GRAND CHOCOLATECANDLE,4,2010-12-23 16:49:00,1.45,NaN
461770,539992,21470,FLOWER VINE RAFFIA FOOD COVER,1,2010-12-23 17:41:00,3.75,NaN
461771,539992,22258,FELT FARM ANIMAL RABBIT,1,2010-12-23 17:41:00,1.25,NaN


In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


**Data Cleansing**

In [13]:
df_clean = df.copy()

# Create 'date' column
df_clean['date'] = pd.to_datetime(df_clean['order_date']).dt.date.astype('datetime64[ns]')

# Delete column with no 'customer_id'
df_clean = df_clean[~df_clean['customer_id'].isna()]

# Delete column with no 'product_name'
df_clean = df_clean[~df_clean['product_name'].isna()]

# Make all 'product_name' lowercase
df_clean['product_name'] = df_clean['product_name'].str.lower()

# Delete rows which 'product_code' or 'product_name' contain test
df_clean = df_clean[(~df_clean['product_code'].str.lower().str.contains('test')) |
                    (~df_clean['product_name'].str.contains('test'))]

# Create 'order_status' column with value 'cancelled' for values start with 'c'
# and 'delivered' for values don't start with 'c' in 'order_id'
df_clean['order_status'] = np.where(df_clean['order_id'].str[:1] == 'C', 'cancelled', 'delivered')

# Change negative quantity to positive, as negative indicates cancelled order
df_clean['quantity'] = df_clean['quantity'].abs()

# Delete rows with negative price
df_clean = df_clean[df_clean['price'] > 0]

# Create 'amount' column as result of 'quantity' and 'price' multiplication
df_clean['amount'] = df_clean['quantity'] * df_clean['price']

# Replace 'product_name' of 'product_code' that has multiple product_name with the one that appears most frequently
most_freq_poduct_name = df_clean.groupby(['product_code', 'product_name'], as_index=False).agg(
    order_cnt=('order_id', 'nunique')).sort_values(['product_code', 'product_name'], ascending=[True, False])
most_freq_poduct_name['rank'] = most_freq_poduct_name.groupby('product_code')['order_cnt'].rank(method='first', ascending=False)
most_freq_poduct_name = most_freq_poduct_name[most_freq_poduct_name['rank']==1].drop(columns=['order_cnt', 'rank'])
df_clean = df_clean.merge(most_freq_poduct_name.rename(columns={'product_name': 'most_frequent_product_name'}), how='left', on='product_code')
df_clean['product_name'] = df_clean['most_frequent_product_name']
df_clean = df_clean.drop(columns='most_frequent_product_name')

# Convert 'customer_id' to string
df_clean['customer_id'] = df_clean['customer_id'].astype(str)

In [14]:
# Remove outliers
from scipy import stats

df_clean = df_clean[(np.abs(stats.zscore(df_clean[['quantity', 'amount']]))<3).all(axis=1)]
df_clean = df_clean.reset_index(drop=True)
df_clean

,order_id,product_code,product_name,quantity,order_date,price,customer_id,date,order_status,amount
0,C493411,21539,red retrospot butter dish,1,2010-01-04 09:43:00,4.25,14590.0,2010-01-04,cancelled,4.25
1,493414,21844,red retrospot mug,36,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,91.80
2,493414,21533,retro spot large milk jug,12,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,51.00
3,493414,37508,new england ceramic cake server,2,2010-01-04 10:28:00,2.55,14590.0,2010-01-04,delivered,5.10
4,493414,35001G,hand open shape gold,2,2010-01-04 10:28:00,4.25,14590.0,2010-01-04,delivered,8.50
...,...,...,...,...,...,...,...,...,...,...
358464,539988,84380,set of 3 butterfly cookie cutters,1,2010-12-23 16:06:00,1.25,18116.0,2010-12-23,delivered,1.25
358465,539988,84849D,hot baths soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358466,539988,84849B,fairy soap soap holder,1,2010-12-23 16:06:00,1.69,18116.0,2010-12-23,delivered,1.69
358467,539988,22854,cream sweetheart egg holder,2,2010-12-23 16:06:00,4.95,18116.0,2010-12-23,delivered,9.90


In [15]:
df_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 358469 entries, 0 to 358468
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      358469 non-null  object        
 1   product_code  358469 non-null  object        
 2   product_name  358469 non-null  object        
 3   quantity      358469 non-null  int64         
 4   order_date    358469 non-null  object        
 5   price         358469 non-null  float64       
 6   customer_id   358469 non-null  object        
 7   date          358469 non-null  datetime64[ns]
 8   order_status  358469 non-null  object        
 9   amount        358469 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 27.3+ MB


**<h3>RFM Segmentation</h3>**

Aggregate transaction data to total transaction summary (order), total order value, last order date each user

In [16]:
df_user = df_clean.groupby('customer_id', as_index=False).agg(
    order_count=('order_id', 'nunique'), max_order_date=('date', 'max'),
    total_order_value=('amount', 'sum'))
df_user

,customer_id,order_count,max_order_date,total_order_value
0,12346.0,5,2010-10-04,602.40
1,12608.0,1,2010-10-31,415.79
2,12745.0,2,2010-08-10,723.85
3,12746.0,2,2010-06-30,266.35
4,12747.0,19,2010-12-13,4094.79
...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77
3885,18284.0,2,2010-10-06,486.68
3886,18285.0,1,2010-02-17,427.00
3887,18286.0,2,2010-08-20,941.48


Create column the number of days from last order

In [21]:
# today is considered as the last order day  in the dataset
today = df_clean['date'].max()
df_user['day_since_last_order'] = (today - df_user['max_order_date']).dt.days
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order
0,12346.0,5,2010-10-04,602.40,80
1,12608.0,1,2010-10-31,415.79,53
2,12745.0,2,2010-08-10,723.85,135
3,12746.0,2,2010-06-30,266.35,176
4,12747.0,19,2010-12-13,4094.79,10
...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31
3885,18284.0,2,2010-10-06,486.68,78
3886,18285.0,1,2010-02-17,427.00,309
3887,18286.0,2,2010-08-20,941.48,125


In [22]:
df_user.describe()

,order_count,max_order_date,total_order_value,day_since_last_order
count,3889.000000,3889,3889.000000,3889.000000
mean,5.128568,2010-09-23 04:46:57.793777664,1544.623084,90.800720
min,1.000000,2010-01-05 00:00:00,1.250000,0.000000
25%,1.000000,2010-08-19 00:00:00,296.360000,25.000000
50%,3.000000,2010-10-26 00:00:00,648.200000,58.000000
75%,6.000000,2010-11-28 00:00:00,1585.940000,126.000000
max,163.000000,2010-12-23 00:00:00,71970.390000,352.000000
std,8.499330,NaN,3434.816315,88.873286


Make binning from the number of days since last order, consists of 5 bins with boundaries: min, P20, P40, P60, P80, and max; labeled from 1 to 5 from highest to lowest as 'recency score'

In [23]:
df_user['recency_score'] = pd.cut(df_user['day_since_last_order'],
                                  bins=[df_user['day_since_last_order'].min(),
                                        np.percentile(df_user['day_since_last_order'], 20),
                                        np.percentile(df_user['day_since_last_order'], 40),
                                        np.percentile(df_user['day_since_last_order'], 60),
                                        np.percentile(df_user['day_since_last_order'], 80),
                                        df_user['day_since_last_order'].max()],
                                  labels=[5,4,3,2,1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score
0,12346.0,5,2010-10-04,602.40,80,2
1,12608.0,1,2010-10-31,415.79,53,3
2,12745.0,2,2010-08-10,723.85,135,2
3,12746.0,2,2010-06-30,266.35,176,1
4,12747.0,19,2010-12-13,4094.79,10,5
...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4
3885,18284.0,2,2010-10-06,486.68,78,2
3886,18285.0,1,2010-02-17,427.00,309,1
3887,18286.0,2,2010-08-20,941.48,125,2


In [24]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 182.4+ KB


Make binning from the number of days since last order, consists of 5 bins with boundaries: min, P20, P40, P60, P80, and max; labeled from 1 to 5 from highest to lowest as 'frequency score'

In [26]:
df_user['frequency_score'] = pd.cut(df_user['order_count'],
                                  bins=[0, # change 'df_user['order_count'].min()' to 0 as its initial value is 1 that is the same as 20th percentile
                                        np.percentile(df_user['order_count'], 20),
                                        np.percentile(df_user['order_count'], 40),
                                        np.percentile(df_user['order_count'], 60),
                                        np.percentile(df_user['order_count'], 80),
                                        df_user['order_count'].max()],
                                  labels=[5,4,3,2,1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score
0,12346.0,5,2010-10-04,602.40,80,2,2
1,12608.0,1,2010-10-31,415.79,53,3,5
2,12745.0,2,2010-08-10,723.85,135,2,4
3,12746.0,2,2010-06-30,266.35,176,1,4
4,12747.0,19,2010-12-13,4094.79,10,5,1
...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,2
3885,18284.0,2,2010-10-06,486.68,78,2,4
3886,18285.0,1,2010-02-17,427.00,309,1,5
3887,18286.0,2,2010-08-20,941.48,125,2,4


In [27]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 7 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int64         
 6   frequency_score       3889 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 212.8+ KB


Make binning from the number of days since last order, consists of 5 bins with boundaries: min, P20, P40, P60, P80, and max; labeled from 1 to 5 from highest to lowest as 'monetary score'

In [28]:
df_user['monetary_score'] = pd.cut(df_user['total_order_value'],
                                  bins=[df_user['total_order_value'].min(),
                                        np.percentile(df_user['total_order_value'], 20),
                                        np.percentile(df_user['total_order_value'], 40),
                                        np.percentile(df_user['total_order_value'], 60),
                                        np.percentile(df_user['total_order_value'], 80),
                                        df_user['total_order_value'].max()],
                                  labels=[5,4,3,2,1],
                                  include_lowest=True).astype(int)
df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score
0,12346.0,5,2010-10-04,602.40,80,2,2,3
1,12608.0,1,2010-10-31,415.79,53,3,5,4
2,12745.0,2,2010-08-10,723.85,135,2,4,3
3,12746.0,2,2010-06-30,266.35,176,1,4,4
4,12747.0,19,2010-12-13,4094.79,10,5,1,1
...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,2,3
3885,18284.0,2,2010-10-06,486.68,78,2,4,3
3886,18285.0,1,2010-02-17,427.00,309,1,5,4
3887,18286.0,2,2010-08-20,941.48,125,2,4,2


In [29]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int64         
 6   frequency_score       3889 non-null   int64         
 7   monetary_score        3889 non-null   int64         
dtypes: datetime64[ns](1), float64(1), int64(5), object(1)
memory usage: 243.2+ KB


Create column 'Segment' based on recency and frequency score

In [33]:
df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating'],
    default='Others')

df_user

,customer_id,order_count,max_order_date,total_order_value,day_since_last_order,recency_score,frequency_score,monetary_score,segment
0,12346.0,5,2010-10-04,602.40,80,2,2,3,10-Hibernating
1,12608.0,1,2010-10-31,415.79,53,3,5,4,02-Loyal Customers
2,12745.0,2,2010-08-10,723.85,135,2,4,3,08-At Risk
3,12746.0,2,2010-06-30,266.35,176,1,4,4,08-At Risk
4,12747.0,19,2010-12-13,4094.79,10,5,1,1,06-New Customers
...,...,...,...,...,...,...,...,...,...
3884,18283.0,6,2010-11-22,641.77,31,4,2,3,03-Potential Loyalists
3885,18284.0,2,2010-10-06,486.68,78,2,4,3,08-At Risk
3886,18285.0,1,2010-02-17,427.00,309,1,5,4,04-Can't Lose Them
3887,18286.0,2,2010-08-20,941.48,125,2,4,2,08-At Risk


In [34]:
df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3889 entries, 0 to 3888
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   customer_id           3889 non-null   object        
 1   order_count           3889 non-null   int64         
 2   max_order_date        3889 non-null   datetime64[ns]
 3   total_order_value     3889 non-null   float64       
 4   day_since_last_order  3889 non-null   int64         
 5   recency_score         3889 non-null   int64         
 6   frequency_score       3889 non-null   int64         
 7   monetary_score        3889 non-null   int64         
 8   segment               3889 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(2)
memory usage: 273.6+ KB


Show RFM Segmentation summary as the total user, mean & median of toal order, total order value, and total days since last order

In [37]:
summary = pd.pivot_table(df_user, index='segment',
                         values=['customer_id', 'day_since_last_order', 'order_count', 'total_order_value'],
                         aggfunc={'customer_id': pd.Series.nunique,
                                  "day_since_last_order": [np.mean, np.median],
                                  'order_count': [np.mean, np.median],
                                  'total_order_value': [np.mean, np.median]})
summary['percentage_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

/tmp/ipython-input-4112900795.py:1: FutureWarning: The provided callable <function mean at 0x7bfe2a316700> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  summary = pd.pivot_table(df_user, index='segment',
/tmp/ipython-input-4112900795.py:1: FutureWarning: The provided callable <function median at 0x7bfe2a27fd80> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  summary = pd.pivot_table(df_user, index='segment',


customer_id day_since_last_order        order_count  \
                           nunique                 mean median        mean   
segment                                                                      
01-Champion                    132            12.689394   13.0    1.621212   
02-Loyal Customers             640            46.739063   44.0    1.457812   
03-Potential Loyalists         617            21.936791   22.0    4.781199   
04-Can't Lose Them             728           207.394231  214.0    1.000000   
05-Need Attention              176            58.613636   59.0    3.397727   
06-New Customers               385            10.137662    9.0   19.493506   
07-Promising                   174            29.936782   29.0   11.965517   
08-At Risk                     627           160.084530  141.0    2.676236   
09-About to Sleep              215            57.134884   58.0    8.283721   
10-Hibernating                 195           127.861538  108.0    7.569231   

                              total_order_value           percentage_unique  
                       median              mean    median                    
segment                                                                      
01-Champion               2.0        470.097727   326.300               3.4  
02-Loyal Customers        1.0        427.805828   330.160              16.5  
03-Potential Loyalists    5.0       1350.968023  1091.300              15.9  
04-Can't Lose Them        1.0        267.103821   201.365              18.7  
05-Need Attention         3.0        989.232676   826.370               4.5  
06-New Customers         14.0       6387.841727  3785.760               9.9  
07-Promising             11.0       3836.533345  2872.100               4.5  
08-At Risk                2.0        699.920990   566.900              16.1  
09-About to Sleep         7.0       2324.219353  1831.880               5.5  
10-Hibernating            6.0       2070.016672  1641.300               5.0